# Bibliotecas

In [1]:
import os, re, json, math
from pathlib import Path
from typing import List, Dict, Any, Tuple

import numpy as np
from tqdm.auto import tqdm

from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import CharacterTextSplitter
import faiss

## 1. Configuração

- `PDF_PATH`: caminho do seu PDF
- `CHUNK_SIZE`: tamanho aproximado do chunk em caracteres (simples e eficiente)
- `CHUNK_OVERLAP`: overlap em caracteres
- `MODEL_NAME`: modelo de embeddings (padrão: `all-MiniLM-L6-v2`)


In [24]:
PDF_PATH = "./dd-5e-livro-do-jogador-fundo-branco-biblioteca-elfica.pdf"  
OUT_DIR = "faiss_store"

CHUNK_SIZE = 512
CHUNK_OVERLAP = 51

MODEL_NAME = "alfaneo/bertimbau-base-portuguese-sts" 

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)


## 2) Leitura do PDF e extração de texto

A extração é feita por página. Guardamos também o número da página para metadados.


In [8]:
def read_pdf_by_page(pdf_path: str) -> List[Dict[str, Any]]:
    pdf_path = str(pdf_path)
    reader = PdfReader(pdf_path)
    pages = []
    for i, page in tqdm(enumerate(reader.pages)):
        txt = page.extract_text() or ""
        if txt:
            pages.append({"page": i + 1, "text": txt})
    return pages

pages = read_pdf_by_page(PDF_PATH)
len(pages)

0it [00:00, ?it/s]

315

## 3) Chunking (separador de chunks)

Este splitter é simples: junta páginas e corta por caracteres com overlap.
Se você preferir splitter por tokens, dá para trocar depois.


In [25]:
def chunk_text(pages: List[Dict[str, Any]], chunk_size: int, overlap: int) -> List[Dict[str, Any]]:
    chunks = []
    for p in pages:
        text = p["text"]
        start = 0
        n = len(text)
        while start < n:
            end = min(start + chunk_size, n)
            chunk = text[start:end].strip()
            if chunk:
                chunks.append({
                    "page": p["page"],
                    "start_char": start,
                    "end_char": end,
                    "text": chunk
                })
            if end == n:
                break
            start = max(0, end - overlap)
    return chunks

chunks = chunk_text(pages, CHUNK_SIZE, CHUNK_OVERLAP)
len(chunks)


3027

## 4) Embeddings

Geramos embeddings para cada chunk.
- Usaremos **cosine similarity** (IndexFlatIP) com vetores normalizados.


In [26]:
model = SentenceTransformer(MODEL_NAME)

texts = [c["text"] for c in chunks]
emb = model.encode(
    texts,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True, 
).astype("float32")

emb.shape, emb.dtype


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: alfaneo/bertimbau-base-portuguese-sts
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Batches:   0%|          | 0/48 [00:00<?, ?it/s]

((3027, 768), dtype('float32'))

## 5) Criar FAISS index (CPU) e persistir

Salvaremos:
- `index.faiss` (vetores)
- `meta.jsonl` (metadados por linha, com o mesmo id do vetor)
- `config.json` (configs úteis)


In [27]:
dim = emb.shape[1]
index = faiss.IndexFlatIP(dim)  # inner product (com vetores normalizados => cosine)
index.add(emb)

index.ntotal


3027

In [28]:
# Persistência
index_path = str(Path(OUT_DIR) / "index.faiss")
meta_path = str(Path(OUT_DIR) / "meta.jsonl")
cfg_path = str(Path(OUT_DIR) / "config.json")

faiss.write_index(index, index_path)

with open(meta_path, "w", encoding="utf-8") as f:
    for i, c in enumerate(chunks):
        rec = {
            "id": i,
            "page": c["page"],
            "start_char": c["start_char"],
            "end_char": c["end_char"],
            "text": c["text"],
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")

with open(cfg_path, "w", encoding="utf-8") as f:
    json.dump({
        "pdf_path": PDF_PATH,
        "chunk_size": CHUNK_SIZE,
        "chunk_overlap": CHUNK_OVERLAP,
        "model_name": MODEL_NAME,
        "dim": dim,
        "faiss_index": "IndexFlatIP (cosine via normalized embeddings)",
    }, f, ensure_ascii=False, indent=2)

(index_path, meta_path, cfg_path)


('faiss_store/index.faiss',
 'faiss_store/meta.jsonl',
 'faiss_store/config.json')

## 6) Busca / Recuperação (RAG básico)

Funções para:
- carregar index e metadados
- consultar por `query` e retornar top-k chunks


In [29]:
def load_meta(meta_jsonl: str) -> List[Dict[str, Any]]:
    items = []
    with open(meta_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            items.append(json.loads(line))
    return items

def load_store(out_dir: str):
    out_dir = Path(out_dir)
    idx = faiss.read_index(str(out_dir / "index.faiss"))
    meta = load_meta(str(out_dir / "meta.jsonl"))
    return idx, meta

def search(query: str, top_k: int = 5, out_dir: str = OUT_DIR):
    idx, meta = load_store(out_dir)
    q_emb = model.encode([query], normalize_embeddings=True, convert_to_numpy=True).astype("float32")
    scores, ids = idx.search(q_emb, top_k)
    results = []
    for score, _id in zip(scores[0], ids[0]):
        if _id == -1:
            continue
        m = meta[int(_id)]
        results.append({
            "score": float(score),
            "id": int(_id),
            "page": m["page"],
            "text": m["text"],
        })
    return results

In [30]:
results = search("O que e um druida?", top_k=3)
for r in results:
    print(f"Score: {r['score']:.4f} | Page: {r['page']}\n{r['text']}\n---\n")

Score: 0.6521 | Page: 74
, ignorando 
barreiras políticas. Todos os druidas são nominalmente 
membros de uma sociedade druídica, apesar de alguns 
indivíduos serem tão isolados que eles nunca chegaram a 
ver membros de alta patente da sociedade ou 
participaram de encontros druídicos. Os druidas 
consideram-se irmãos e irmãs. Como criaturas na 
natureza, no entanto, os druidas, as vezes, competem, ou 
mesmo caçam uns aos outros. 
Em uma escala local, os druidas são organizados em 
círculos que partilham de certas perspectivas de 
n
---

Score: 0.6056 | Page: 136
nado por cerveja, vinho e outras 
bebidas. 
2 Não existe lugar para precaução em uma vida vivida ao 
máximo. 
3 Eu lembro de cada insulto que sofri e nutro um 
ressentimento silencioso contra qualquer um que já tenha 
me insultado 
4 Eu tenho dificuldade em confiar em membros de outras 
raças, tribos ou sociedades. 
5 A violência é minha resposta para quase todos os 
obstáculos. 
6 Não espere que eu salve aqueles que não conseg